# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.121265e+02     1.756276e+00
 * time: 0.9715509414672852
     1     1.077159e+01     8.753044e-01
 * time: 3.088265895843506
     2    -1.174062e+01     9.809496e-01
 * time: 3.979987859725952
     3    -3.431387e+01     7.147192e-01
 * time: 4.786112070083618
     4    -4.795260e+01     5.279254e-01
 * time: 5.5951550006866455
     5    -5.715239e+01     1.831252e-01
 * time: 6.417973041534424
     6    -5.993499e+01     8.191751e-02
 * time: 6.952760934829712
     7    -6.087652e+01     5.579144e-02
 * time: 7.490429878234863
     8    -6.125164e+01     5.177226e-02
 * time: 8.041635036468506
     9    -6.156147e+01     4.828545e-02
 * time: 8.578257083892822
    10    -6.177256e+01     3.512589e-02
 * time: 9.116736888885498
    11    -6.192424e+01     2.533055e-02
 * time: 9.663635015487671
    12    -6.200585e+01     2.186475e-02
 * time: 10.20151400566101
    13    -6.208105e+01     1.961405e-02
 * time: 10.740378856658936
    

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671088
    AtomicLocal         -18.8557707
    AtomicNonlocal      14.8522645
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485388 
    Xc                  -19.3336827

    total               -62.261666456151
